## Install/load required packages

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install(c("DESeq2", "qvalue", "AnnotationDbi", "DOSE"), force = TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.12 (BiocManager 1.30.16), R 4.0.1 (2020-06-06)

Installing package(s) 'BiocVersion', 'DESeq2', 'qvalue', 'AnnotationDbi',
  'DOSE'

also installing the dependencies ‘RCurl’, ‘matrixStats’, ‘lambda.r’, ‘XML’, ‘xtable’, ‘GenomeInfoDb’, ‘XVector’, ‘MatrixGenerics’, ‘DelayedArray’, ‘futile.logger’, ‘BH’, ‘annotate’, ‘survival’, ‘plogr’, ‘gridExtra’, ‘fastmatch’, ‘GO.db’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘BiocGenerics’, ‘Biobase’, ‘BiocParallel’, ‘genefilter’, ‘locfit’, ‘geneplotter’, ‘RcppArmadillo’, ‘RSQLite’, ‘DO.db’, ‘fgsea’, ‘GOSemSim’




In [ ]:
library(DESeq2)
library(tidyverse)
library(qvalue)
library(AnnotationDbi)
library(DOSE)


## Read in the data

### Read in the raw read counts data

In [ ]:
TranscriptData <- as.matrix(read.csv("RNAseq.csv", row.names = "transcript_id"))

### Read in the data table of sample information

In [ ]:
ColData <- read.csv("Sample_Names.txt",  sep="\t", row.names=1)

### Confirm that the transcript data row names match the coldata column names

In [ ]:
all(rownames(ColData) %in% colnames(TranscriptData))
all(rownames(ColData) == colnames(TranscriptData))

## Make DESeq Dataset 

### Generate DESeq Dataset from read counts matrix

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = TranscriptData, colData = ColData)

## Starting number of genes:
nrow(dds) 

#### Pre-filter for any rows that have less then 10 counts across them

In [ ]:
keep <- rowSums(counts(dds)) >= 10
## e.g.: keep <- rowSums(counts(dds) >= 10) >= 3, this would be at least 3 samples with 10 reads.

dds <- dds[keep,]

## Remaining number of genes:
nrow(dds) 

## Normalize/Transform DESeq dataset

#### This returns a DESeqTransform object. Stored data is no longer counts and must be accessed through assay. Sequencing depth normalization is done automatically


In [ ]:
vsd <- vst(dds, blind = FALSE) 
## blind = FALSE The experimental design is not used directly in the transformation, only in estimating the global amount of variability in the counts.

head(assay(vsd), 3)
colData(vsd)

rld <- rlog(dds, blind = FALSE)
head(assay(rld), 3)
## rlog supposed to work better with smaller sample size and when there is more variation in sequencing depth


## Differential Expression Analysis

In [ ]:
dds_analysis <- DESeq(dds)

### JG-1 vs. DMSO

In [ ]:
resJG1 <- results(dds_analysis, contrast = c("Condition", "JG1", "DMSO"))
## contrast takes specified groups from a factored variable in the data table ("Condition"), and calculates DE of numerator ("JG1") vs. demoninator ("DMSO)

resJG1

## Number of differentially expressed genes with significant p-value:
sum(resJG1$pvalue < 0.05, na.rm=TRUE) 

## Number of differentially expressed genes that are missing a p-value:
sum(is.na(resJG1$pvalue)) 

### M4 vs. DMSO

In [ ]:
resM4 <- results(dds_analysis, contrast = c("Condition", "M4", "DMSO")) 
resM4
sum(resM4$pvalue < 0.05, na.rm=TRUE) 
sum(is.na(resM4$pvalue)) 

# Export results

In [ ]:
resJG1Ordered <- resJG1[order(resJG1$pvalue),]
## Orders data by pvalue

resJG1OrderedDF <- as.data.frame(resJG1Ordered)
resJG1OrderedDF <- remove_missing(resJG1OrderedDF, vars = "padj")
## Makes a dataframe and removes rows with missing padj

write.csv(resJG1OrderedDF, file = "JG-1 DE Results.csv")

In [ ]:
resM4Ordered <- resM4[order(resM4$pvalue),]
resM4OrderedDF <- as.data.frame(resM4Ordered)
resM4OrderedDF <- remove_missing(resM4OrderedDF, vars = "padj")

write.csv(resM4OrderedDF, file = "M4 DE Results.csv")